<a href="https://colab.research.google.com/github/JangJiYeon12/AI-Project/blob/main/Section4_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 테스트 데이터셋을 구글에서 '삼색이'를 검색했을 때 나오는 이미지들을 크롤링하는 함수입니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

from selenium import webdriver
import os
import time
from tqdm import tqdm

     |████████████████████████████████| 979 kB 4.1 MB/s 
     |████████████████████████████████| 138 kB 45.8 MB/s 
     |████████████████████████████████| 359 kB 42.8 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 4.0 MB 41.8 MB/s 
     |████████████████████████████████| 58 kB 4.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-

In [ ]:


### step 1. 이미지 크롤링

# 저장된 이미지 제거
#!rm -rf "/content/images" #다운로드 받은 이미지 폴더(base_dir의 주소) 전체 제거
#!rm -rf "/content/images/폴더명" #다운로드 받은 이미지 폴더에서 "폴더명"에 해당하는 폴더 전체 제거

# 크롤링 할 검색어, 이미지 개수
keyword = input("크롤링할 검색어 입력: ")
image_num = int(input("크롤링할 이미지 개수: "))

# 크롬창 띄우기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get("https://www.google.com/search?q=%s&source=lnms&tbm=isch&sa=X&ved=2ahUKEwj_weCY6M7xAhUHhZQKHRgRDy0Q_AUoAXoECAEQAw&biw=1920&bih=937"%keyword)

# 첫 번째 사진 클릭
driver.find_element_by_css_selector(".rg_i.Q4LuWd").click()

# url 수집
img_url = []
for i in tqdm(range(image_num)):
    # 이미지 url get
    url = driver.find_element_by_css_selector(".n3VNCb").get_attribute("src")
    img_url.append(url)

    # 다음 이미지
    if i == 0:
        element = driver.find_element_by_css_selector(".gvi3cf")
        driver.execute_script("arguments[0].click();", element)
    element = driver.find_element_by_css_selector(".gvi3cf")
    driver.execute_script("arguments[0].click();", element)

# 크롬창 종료
driver.close()

print('수집된 url 개수 :', len(img_url))

# 저장된 이미지 제거
#!rm -rf "/content/images" #다운로드 받은 이미지 폴더(base_dir의 주소) 전체 제거
#!rm -rf "/content/images/폴더명" #다운로드 받은 이미지 폴더에서 "폴더명"에 해당하는 폴더 전체 제거

# HTTP Error 403 해결
import random
import urllib.request

opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
urllib.request.install_opener(opener)

# 폴더 생성
if not os.path.exists("images"):
    os.makedirs("images")

if not os.path.exists("./images/"+keyword):
    os.makedirs("./images/"+keyword)

# 이미지 저장
count = 0
err_img_url = []
for n, i in enumerate(img_url):
    try:
        urllib.request.urlretrieve(i, "./images/{0}/{0}_{1}.jpg".format(keyword, count))
        count += 1
        if n!=0 and n%50==0:
            print(n, "/", len(img_url), "개 완료")
    except:
        err_img_url.append(i)
        print("error 발생으로 수집안됨 (번호: %d)" % n)   # 주로 503 error, error 발생시 수동으로 이미지 저장 (  img_url[error No.]  )
print('\n저장된 이미지 개수 :', count, '\n')
print('에러 이미지 URL:')
for i in err_img_url:
    print(i)

# 크롤링한 이미지 폴더 압축
import zipfile
 
fantasy_zip = zipfile.ZipFile('/content/crawler_google_image.zip', 'w') # 압축 후 저장 위치
for folder, subfolders, files in os.walk('/content/images'): # 압축 할 폴더
    for file in files:
        if file.endswith('.jpg'): # 압축할 파일 확장자
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), '/content/images/'), compress_type = zipfile.ZIP_DEFLATED)
fantasy_zip.close()

# 압축 파일 구글 드라이브에 복사
!cp '/content/crawler_google_image.zip' '/content/drive/MyDrive/Colab Notebooks/crawler_google_image.zip'